In [1]:
import tensorflow as tf
from sklearn.decomposition import PCA
from tensorflow.keras.preprocessing.text import Tokenizer
import re
from tensorflow.keras.layers import Dense,Embedding,Lambda
import numpy as np
from tensorflow.keras.models import Sequential
# CBOW(Ass5)/CBOW.txt

In [2]:
with open('../LP-IV-datasets/CBOW(Ass5)/CBOW.txt','r') as file:
    data = file.read()

In [5]:
clean_sentence = [re.sub('[^A-Za-z0-9]+',' ',sentence.lower().strip()) for sentence in data.split('.') if sentence]
tokenizer = Tokenizer()
tokenizer.fit_on_texts(clean_sentence)
sequences = tokenizer.texts_to_sequences(clean_sentence)
clean_sentence



['the speed of transmission is an important point of difference between the two viruses',
 'influenza has a shorter median incubation period the time from infection to appearance of symptoms and a shorter serial interval the time between successive cases than covid 19 virus',
 'the serial interval for covid 19 virus is estimated to be 5 6 days while for influenza virus the serial interval is 3 days',
 'this means that influenza can spread faster than covid 19',
 'further transmission in the first 3 5 days of illness or potentially pre symptomatic transmission transmission of the virus before the appearance of symptoms is a major driver of transmission for influenza',
 'in contrast while we are learning that there are people who can shed covid 19 virus 24 48 hours prior to symptom onset at present this does not appear to be a major driver of transmission',
 'the reproductive number the number of secondary infections generated from one infected individual is understood to be between 2 an

In [28]:
word_to_index= {word:index for word,index in tokenizer.word_index.items()}
index_to_word= {index:word for word,index in tokenizer.word_index.items()}

In [29]:
vocab_size = len(tokenizer.word_index) + 1
embedding_size = 10
window_size=2

In [31]:
contexts = [ sequence[i-window_size:i]+ sequence[i+1:i+window_size+1] for sequence in sequences 
            for i in range(window_size,len(sequence)-window_size)]
target = [ sequence[i] for sequence in sequences 
            for i in range(window_size,len(sequence)-window_size)]

In [32]:
X = np.array(contexts)
Y = np.array(target)

In [39]:
model = Sequential([
    Embedding(input_dim = vocab_size,output_dim = embedding_size,input_length=window_size*2),
    Lambda(lambda x:tf.reduce_mean(x,axis = 1)),
    Dense(256, activation='relu'),
    Dense(512, activation='relu'),
    Dense(units=vocab_size, activation='softmax')
])

In [40]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
model.fit(X,Y,epochs=200)

Epoch 1/200
5/5 [==============================] - 12s 49ms/step - loss: 4.6154 - accuracy: 0.0252
Epoch 2/200
5/5 [==============================] - 0s 47ms/step - loss: 4.6035 - accuracy: 0.0503
Epoch 3/200
5/5 [==============================] - 0s 28ms/step - loss: 4.5893 - accuracy: 0.0629
Epoch 4/200
5/5 [==============================] - 0s 27ms/step - loss: 4.5619 - accuracy: 0.0566
Epoch 5/200
5/5 [==============================] - 0s 44ms/step - loss: 4.5122 - accuracy: 0.0566
Epoch 6/200
5/5 [==============================] - 0s 40ms/step - loss: 4.4353 - accuracy: 0.0503
Epoch 7/200
5/5 [==============================] - 0s 28ms/step - loss: 4.3314 - accuracy: 0.0503
Epoch 8/200
5/5 [==============================] - 0s 45ms/step - loss: 4.2340 - accuracy: 0.0566
Epoch 9/200
5/5 [==============================] - 0s 48ms/step - loss: 4.1953 - accuracy: 0.0755
Epoch 10/200
5/5 [==============================] - 0s 43ms/step - loss: 4.1537 - accuracy: 0.0503
Epoch 11/200
5/5 [

5/5 [==============================] - 0s 44ms/step - loss: 0.0080 - accuracy: 1.0000
Epoch 166/200
5/5 [==============================] - 0s 41ms/step - loss: 0.0079 - accuracy: 1.0000
Epoch 167/200
5/5 [==============================] - 0s 37ms/step - loss: 0.0077 - accuracy: 1.0000
Epoch 168/200
5/5 [==============================] - 0s 36ms/step - loss: 0.0076 - accuracy: 1.0000
Epoch 169/200
5/5 [==============================] - 0s 34ms/step - loss: 0.0075 - accuracy: 1.0000
Epoch 170/200
5/5 [==============================] - 0s 37ms/step - loss: 0.0073 - accuracy: 1.0000
Epoch 171/200
5/5 [==============================] - 0s 41ms/step - loss: 0.0072 - accuracy: 1.0000
Epoch 172/200
5/5 [==============================] - 0s 54ms/step - loss: 0.0070 - accuracy: 1.0000
Epoch 173/200
5/5 [==============================] - 0s 35ms/step - loss: 0.0069 - accuracy: 1.0000
Epoch 174/200
5/5 [==============================] - 0s 48ms/step - loss: 0.0068 - accuracy: 1.0000
Epoch 175/200


In [45]:
embedding = model.get_weights()[0]

In [47]:
pca = PCA(n_components=2)
reduced_emb = pca.fit_transform(embedding)

In [51]:
test_sen = [" speed of is "]
x_test = [tokenizer.texts_to_sequences(test_sen)[0] + [0]*(2*window_size - len(tokenizer.texts_to_sequences(test_sen)[0]))]
test_pred = model.predict(x_test)
y_pred = np.argmax(test_pred)
print(test_sen[0].split() , index_to_word.get(y_pred))

1/1 [==============================] - 0s 204ms/step
['speed', 'of', 'is'] transmission
